## PREPARAR DATA NO CRYPTO

In [11]:
import pandas as pd

# Leer los tres archivos CSV
df1 = pd.read_csv('nytimes_2009_3.csv')
df2 = pd.read_csv('nytimes_2010_5.csv')
df3 = pd.read_csv('nytimes_2009_6.csv')

# Concatenar los DataFrames
df_combinado = pd.concat([df1, df2, df3], ignore_index=True)



# Guardar el DataFrame filtrado en un nuevo archivo CSV llamado 'model_crypto_data.csv'
df_combinado.to_csv('model_crypto_no_data.csv', index=False)

print("¡Archivo filtrado y guardado como 'model_crypto_data.csv' con éxito!")


¡Archivo filtrado y guardado como 'model_crypto_data.csv' con éxito!


## PREPARAR DATA CRYPTO

In [12]:
import pandas as pd
import json

# Cargar datos desde un archivo JSON usando UTF-8
with open('news_2023_bitcoin_cointelegraph.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Convertir la lista de diccionarios en un DataFrame
df = pd.DataFrame(data)

# Remover las columnas 'link' y 'query'
df = df.drop(columns=['link', 'query'])

# Guardar el DataFrame como un archivo CSV
df.to_csv('model_crypto_data.csv', index=False)


## FUSIONAR DF Y AÑADIR LABELS

In [17]:
import pandas as pd

# Cargar los datasets
df_no_crypto = pd.read_csv('model_crypto_no_data.csv', usecols=['Title'])  # Cambia 'texto' por el nombre real de la columna
df_crypto = pd.read_csv('model_crypto_data.csv', usecols=['title'])  # Cambia 'texto' por el nombre real de la columna

# Etiquetar los datos
df_no_crypto['label'] = 0
df_crypto['label'] = 1

# Renombrar la columna de texto en ambos datasets para que tengan el mismo nombre
df_no_crypto.rename(columns={'Title': 'text'}, inplace=True)
df_crypto.rename(columns={'title': 'text'}, inplace=True)

# Combinar los dos datasets en uno solo
df = pd.concat([df_no_crypto, df_crypto])

# Manejar valores NaN: eliminar filas con NaN o rellenarlas con una cadena vacía
df.dropna(subset=['text'], inplace=True)

# Verificar que los datos estén combinados correctamente
print(df.head())


                                                text  label
0            Legal Battles Await Olmert as Term Ends      0
1  Despite the Recession, Neediest Cases Fund Col...      0
2  Raising a Daughter to Thrive on Her Own, Then ...      0
3  In a Life of Hard Knocks, a Mother With an Iro...      0
4                              Beware the False Dawn      0


## TRAIN & RESULTS

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Vectorización del texto usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X_vectorized = vectorizer.fit_transform(df['text'])

# Separar las características (X) y la etiqueta (y)
X = X_vectorized
y = df['label']

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Imprimir la precisión y otras métricas de evaluación
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
print(confusion_matrix(y_test, y_pred))
# Prueba con un nuevo texto
new_text = ["Amazing new Altcoin going up!"]
new_text_tfidf = vectorizer.transform(new_text)
prediction = model.predict(new_text_tfidf)
print("Predicción:", "Relacionado con criptomonedas" if prediction[0] == 1 else "No relacionado con criptomonedas")


Accuracy: 0.9737965260545905
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      7107
           1       0.94      0.97      0.96      2968

    accuracy                           0.97     10075
   macro avg       0.96      0.97      0.97     10075
weighted avg       0.97      0.97      0.97     10075

[[6926  181]
 [  83 2885]]
Predicción: Relacionado con criptomonedas


## TEST CON DATA

In [26]:
import pandas as pd

# Cargar el nuevo dataset
new_texts_tfidf = pd.read_csv('test_model_P1.csv', usecols=['cleaned_text'])  # Cambia 'cleaned_text' por el nombre real de la columna de texto

# Vectorizar el texto del nuevo dataset usando el vectorizador entrenado
new_texts_vectorized = vectorizer.transform(new_texts_tfidf['cleaned_text'])

# Hacer predicciones en el nuevo dataset
new_predictions = model.predict(new_texts_vectorized)

# Añadir las predicciones al DataFrame original
new_texts_tfidf['prediction'] = new_predictions
new_texts_tfidf['prediction_label'] = new_texts_tfidf['prediction'].apply(lambda x: "Relacionado con criptomonedas" if x == 1 else "No relacionado con criptomonedas")

# Guardar el DataFrame con las predicciones en un archivo CSV
output_file = 'predicciones.csv'
new_texts_tfidf.to_csv(output_file, index=False)

print(f"Predicciones guardadas en el archivo {output_file}")


Predicciones guardadas en el archivo predicciones.csv


### TESTS

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Vectorización de textos
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Entrenamiento del modelo de clasificación
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Prueba con un nuevo texto
new_text = ["I am investing in Dogecoin"]
new_text_tfidf = vectorizer.transform(new_text)
prediction = model.predict(new_text_tfidf)
print("Predicción:", "Relacionado con criptomonedas" if prediction[0] == 1 else "No relacionado con criptomonedas")


Accuracy: 0.9935483870967742
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7107
           1       1.00      0.98      0.99      2968

    accuracy                           0.99     10075
   macro avg       1.00      0.99      0.99     10075
weighted avg       0.99      0.99      0.99     10075

Predicción: No relacionado con criptomonedas


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


# Asegurarse de que X_train y X_test son listas de texto
X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label'], test_size=0.2, random_state=42)

# Vectorización de textos
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Entrenamiento del modelo SVM
model = SVC(kernel='linear')  # Utilizando el kernel lineal
model.fit(X_train_tfidf, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Prueba con un nuevo texto
new_text = ["Investing in cryptos might not be as OverPowered"]
new_text_tfidf = vectorizer.transform(new_text)
prediction = model.predict(new_text_tfidf)
print("Predicción:", "Relacionado con criptomonedas" if prediction[0] == 1 else "No relacionado con criptomonedas")

Accuracy: 0.99712158808933
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7107
           1       1.00      0.99      1.00      2968

    accuracy                           1.00     10075
   macro avg       1.00      1.00      1.00     10075
weighted avg       1.00      1.00      1.00     10075

Predicción: No relacionado con criptomonedas


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entrenamiento del modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Prueba con un nuevo texto
new_text = ["I am investing in Dogecoin"]
new_text_tfidf = vectorizer.transform(new_text)
prediction = model.predict(new_text_tfidf)
print("Predicción:", "Relacionado con criptomonedas" if prediction[0] == 1 else "No relacionado con criptomonedas")

Accuracy: 0.9979156327543425
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7107
           1       1.00      0.99      1.00      2968

    accuracy                           1.00     10075
   macro avg       1.00      1.00      1.00     10075
weighted avg       1.00      1.00      1.00     10075

Predicción: No relacionado con criptomonedas
